<a href="https://colab.research.google.com/github/dunomi/zw/blob/master/z9a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ConfirmedCases_raw=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/virus_cases.csv')
USA_state_names= pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/states.csv')
USA_humidity_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_humidity.csv?token=AO3BJXHYBDBENHN2JIXXFGC6POTP2')
USA_temperature_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_temperature.csv?token=AO3BJXFW6K5G6GYWHX5I2GC6POTVA')

In [0]:
USA=ConfirmedCases_raw[ConfirmedCases_raw['Country/Region']=='US'].reset_index().drop(['index','Country/Region','Lat','Long'],axis=1)
#USA.sum(axis = 0, skipna = True)[1:]>10
USA=USA.head(52)
USA=USA[~USA['Province/State'].isin(['Diamond Princess','Grand Princess'])].reset_index().drop('index',axis=1)
USA

In [9]:
USAcases=pd.DataFrame(columns=['Stan','Przyrosty','Data'])

for i in range(len(USA)):
  Date=USA.loc[i][USA.loc[i][0:]!=0].index[1]
  stateName=USA.loc[i][0]
  stateValue=USA.loc[i][1:].tolist()
  stateSum=[i for i in stateValue if i>0]
  stateSumArray=np.array(stateSum[1:7])/np.array(stateSum[0:6])
  stateSumArray=np.array([round(i,2) for i in stateSumArray])
  USAcases=USAcases.append(pd.DataFrame([[stateName,stateSumArray,Date]],columns=['Stan','Przyrosty','Data']))

USAcases=USAcases.reset_index().drop('index',axis=1)[['Stan','Data','Przyrosty']]


,Stan,Data,Przyrosty
0,Washington,3/10/20,"[1.37, 1.21, 1.29, 1.01, 1.12, 1.41]"
1,New York,3/10/20,"[1.27, 1.49, 1.28, 1.25, 1.39, 1.32]"
2,California,3/10/20,"[1.23, 1.25, 1.28, 1.21, 1.25, 1.31]"
3,Massachusetts,3/10/20,"[1.03, 1.14, 1.14, 1.12, 1.19, 1.2]"
4,Georgia,3/10/20,"[1.35, 1.35, 1.35, 1.57, 1.5, 1.22]"
5,Colorado,3/10/20,"[2.27, 1.32, 1.09, 2.06, 1.3, 1.22]"
6,Florida,3/10/20,"[1.87, 1.25, 1.43, 1.52, 1.51, 1.35]"
7,New Jersey,3/10/20,"[1.53, 1.26, 1.0, 2.38, 1.42, 1.82]"
8,Oregon,3/10/20,"[1.27, 1.26, 1.25, 1.07, 1.12, 1.08]"
9,Texas,3/10/20,"[1.62, 1.29, 1.59, 1.33, 1.26, 1.18]"


In [0]:
USAweather=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][3:])
  wilg=np.array(USA_humidity_row.loc[i][3:])/100

  USAweather=USAweather.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))
  
USAdata=pd.merge(USAcases,USAweather)

In [0]:
USAcumdata=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata)):
  stateName=USAdata.loc[i][0]
  przy=np.cumprod(USAdata['Przyrosty'][i])[5]
  temp=USAdata.Temperatura[i].mean()
  wilg=USAdata.Wilgotnosc[i].mean()
  USAcumdata=USAcumdata.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

#USAcumdata.loc[USAcumdata.Przyrosty > 50, 'Przyrosty'] = max(USAcumdata.loc[USAcumdata.Przyrosty < 50].Przyrosty)

In [0]:
for i in range(len(USAdata)):
  USAdata.loc[i, 'Temperatura_cor']=np.corrcoef(USAdata.Przyrosty[i],USAdata.Temperatura[i][0:6].astype(float))[0,1]
  USAdata.loc[i, 'Wilgotnosc_cor']=np.corrcoef(USAdata.Przyrosty[i],USAdata.Wilgotnosc[i][0:6].astype(float))[0,1]

  USAdata.loc[i, 'Temperatura_cor_wsp_ols']=sm.OLS(USAdata['Przyrosty'][i].tolist(),sm.add_constant(USAdata['Temperatura'][i][0:6].tolist())).fit().params[1]
  USAdata.loc[i, 'Wilgotnosc_cor_wsp_ols']=sm.OLS(USAdata['Przyrosty'][i].tolist(),sm.add_constant(USAdata['Wilgotnosc'][i][0:6].tolist())).fit().params[1]

  USAdata.loc[i, 'Przyrosty_wykres_osY']=(max(USAdata.Przyrosty[i])-min(USAdata.Przyrosty[i]))/2
  USAdata.loc[i, 'Temperatura_wykres_osX']=(max(USAdata.Temperatura[i])-min(USAdata.Temperatura[i]))/2
  USAdata.loc[i, 'Wilgotnosc_wykres_osX']=(max(USAdata.Wilgotnosc[i])-min(USAdata.Wilgotnosc[i]))/2

In [26]:
USAdata.to_csv(index=True)

from google.colab import files

writer = pd.ExcelWriter('hey.csv')
USAdata.to_csv(writer)
files.download(USAdata.to_csv(index=True))

',Stan,Data,Przyrosty,Temperatura,Wilgotnosc,Temperatura_cor,Wilgotnosc_cor,Temperatura_cor_wsp_ols,Wilgotnosc_cor_wsp_ols,Przyrosty_wykres_osY,Temperatura_wykres_osX,Wilgotnosc_wykres_osX\n0,Washington,3/10/20,[1.37 1.21 1.29 1.01 1.12 1.41],[12 12 11 3 6 8 14],[0.45 0.45 0.46 0.91 0.55 0.38 0.33],0.7002575825824381,-0.8637455486472698,0.02910891089108917,-0.6851117519826964,0.19999999999999996,5.5,0.29000000000000004\n1,New York,3/10/20,[1.27 1.49 1.28 1.25 1.39 1.32],[17 8 11 13 7 8 8],[0.56 0.36 0.4 0.63 0.44 0.27 0.21],-0.6878499294278391,-0.5228741467723614,-0.016409090909090762,-0.3614503816793886,0.12,5.0,0.21000000000000002\n2,California,3/10/20,[1.23 1.25 1.28 1.21 1.25 1.31],[19 22 23 18 14 11 13],[0.45 0.48 0.38 0.58 0.71 0.74 0.63],-0.3460227137090778,0.26578858551801476,-0.002667706708268297,0.06501256281407031,0.050000000000000044,6.0,0.18\n3,Massachusetts,3/10/20,[1.03 1.14 1.14 1.12 1.19 1.2 ],[19 11 6 16 12 5 7],[0.62 0.53 0.77 0.84 0.32 0.44 0.42],-0.7689756801362972

In [0]:
from google.colab import drive
drive.mount('/content/drive')